This Jupyter Notebook builds upon the referenced [Google Colab](https://colab.research.google.com/drive/1QzQYQxH052Rs088rxMQDveRjoVdkdl74?usp=sharing), integrating customized modifications to address the specific requirements of our testing.

# 1. Load Multimodal Data

## 1.1. Extract and Partition Text, Tables, and Images

In [ ]:
# Remove existing figures
!rm -rf ./figures

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader

# Directory containing the PDFs
pdf_dir = './references'

# Collect all PDF files from the directory and subdirectories, excluding hidden ones
pdf_files = []
for root, dirs, files in os.walk(pdf_dir):
    # Exclude hidden directories
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    for file in files:
        # Exclude hidden files
        if file.lower().endswith('.pdf') and not file.startswith('.'):
            pdf_files.append(os.path.join(root, file))

# Initialize an empty list to hold data from all PDFs
data = []

# Loop through each PDF file and load it
for pdf_file in pdf_files:
    print(f'Loading {pdf_file}')
    loader = UnstructuredPDFLoader(
        file_path=pdf_file,
        strategy='hi_res',
        extract_images_in_pdf=True,
        infer_table_structure=True,
        chunking_strategy="by_title",     # section-based chunking
        max_characters=4000,              # max size of chunks
        new_after_n_chars=4000,           # preferred size of chunks
        combine_text_under_n_chars=2000,  # combine smaller chunks
        mode='elements',
        image_output_dir_path='./figures'
    )
    data.extend(loader.load())

In [ ]:
docs = []
tables = []

for doc in data:
    if doc.metadata['category'] == 'Table':
        tables.append(doc)
    elif doc.metadata['category'] == 'CompositeElement':
        docs.append(doc)

len(docs), len(tables)

In [ ]:
import htmltabletomd

for table in tables:
    table.page_content = htmltabletomd.convert_table(table.metadata['text_as_html'])

## 1.2. Examine Loaded Data (Optional)

In [ ]:
from IPython.display import HTML, display, Markdown, Image

In [ ]:
docs[0]

In [ ]:
print(tables[0].page_content)

In [ ]:
tables[0].metadata['text_as_html']

In [ ]:
display(Markdown(tables[0].metadata['text_as_html']))

In [ ]:
for table in tables:
    print(table.page_content)
    print()

In [ ]:
! ls -l ./figures

In [ ]:
Image('./figures/figure-5-4.jpg')

# 2. Connect to LLM through API

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [ ]:
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o', temperature=0)

# 3. Generate Text-Based Summaries for Multimodal Data

## 3.1. Create Text and Table Summaries

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text particularly for semantic retrieval.
These summaries will be embedded and used to retrieve the raw text or table elements.
Give a detailed summary of the table or text below that is well optimized for retrieval.
For any tables also add in a one line description of what the table is about besides the summary.
Do not add additional words like Summary: etc.

Table or text chunk:
{element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
summarize_chain = (
                    {"element": RunnablePassthrough()}
                      |
                    prompt
                      |
                    chatgpt
                      |
                    StrOutputParser() # Extracts the response as text and returns it as a string
)

# Initialize empty summaries
text_summaries = []
table_summaries = []

text_docs = [doc.page_content for doc in docs]
table_docs = [table.page_content for table in tables]

text_summaries = summarize_chain.batch(text_docs, {"max_concurrency": 5})
table_summaries = summarize_chain.batch(table_docs, {"max_concurrency": 5})

len(text_summaries), len(table_summaries)

## 3.2. Examine Text and Table Summaries (Optional)

In [ ]:
text_summaries[0]

In [ ]:
table_summaries[0]

## 3.3. Create Image Summaries

In [ ]:
import base64
from langchain_core.messages import HumanMessage

# Create a function to encode images
def encode_image(image_path):
    # Get the base64 string
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Create image summaries
def image_summarize(img_base64, prompt):
    chat = ChatOpenAI(model="gpt-4o", temperature=0)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval.
                Remember these images could potentially contain graphs, charts or tables also.
                These summaries will be embedded and used to retrieve the raw image for question answering.
                Give a detailed summary of the image that is well optimized for retrieval.
                Do not add additional words like Summary: etc.
             """

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

# Image summaries
IMG_PATH = './figures'
imgs_base64, image_summaries = generate_img_summaries(IMG_PATH)

len(imgs_base64), len(image_summaries)

## 3.4. Examine Image Summaries (Optional)

In [ ]:
display(Image('./figures/figure-1-1.jpg'))

In [ ]:
image_summaries[0]

# 4. Build Multi-Vetor Retrievers: Multimodal and Single-Modal

## 4.1. Access Embedding Model

In [ ]:
from langchain_openai import OpenAIEmbeddings

openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-large')

## 4.2. Create Utility Functions

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.storage import RedisStore
from langchain_community.utilities.redis import get_client
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# Create retriever that indexes summaries, but returns raw images or texts
def create_multi_vector_retriever(
    docstore, vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):

    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=docstore,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

## 4.3. Initiate Vectorstores: Chroma

In [ ]:
# The vectorstore to use to index the summaries and their embeddings
chroma_db_multimodal = Chroma(
    collection_name="mm_rag",
    embedding_function=openai_embed_model,
    collection_metadata={"hnsw:space": "cosine"},
)

# The vectorstore for single-modal RAG
chroma_db_single_modal = Chroma(
    collection_name="text_rag",
    embedding_function=openai_embed_model,
    collection_metadata={"hnsw:space": "cosine"},
)

## 4.4. Initiate Docstores: Redis and InMemoryStore

In JupyterLab's terminal (File > New > Terminal), run the following commands:

```bash
# 1. Import the GPG key for the Redis repository
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

# 2. Add the Redis repository to your sources list
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] \
https://packages.redis.io/deb $(lsb_release -cs) main" | \
sudo tee /etc/apt/sources.list.d/redis.list

# 3. Update package lists
sudo apt-get update

# 4. Install Redis Stack Server
sudo apt-get install redis-stack-server

# 5. Start Redis Stack Server in the background
redis-stack-server --daemonize yes
```

In [ ]:
# Initialize the storage layer - to store raw images, text and tables
client = get_client('redis://localhost:6379')
redis_store = RedisStore(client=client) # Can use filestore, memorystory, any other DB store also

from langchain.storage import InMemoryStore

# Initialize the storage layer for the single-modal retriever
docstore_single_modal = InMemoryStore()

## 4.5. Create Retrievers

In [ ]:
retriever_multimodal = create_multi_vector_retriever(
    redis_store,
    chroma_db_multimodal,
    text_summaries,
    text_docs,
    table_summaries,
    table_docs,
    image_summaries,
    imgs_base64,
)

retriever_multimodal

In [ ]:
retriever_single_modal = create_multi_vector_retriever(
    docstore_single_modal,
    chroma_db_single_modal,
    text_summaries,
    text_docs,
    table_summaries=[],  # Empty lists since we're only using text
    tables=[],
    image_summaries=[],  # No images
    images=[],
)

# Verify the single-modal retriever
retriever_single_modal

# 5. Prepare for Retrieval

## 5.1. Setup Retrieval Display

In [ ]:
from IPython.display import HTML, display, Markdown, Image
from PIL import Image
import base64
from io import BytesIO

# Disply base64 encoded string as image
def plt_img_base64(img_base64):
    # Decode the base64 string
    img_data = base64.b64decode(img_base64)
    # Create a BytesIO object
    img_buffer = BytesIO(img_data)
    # Open the image using PIL
    img = Image.open(img_buffer)
    display(img)

In [ ]:
import re
import base64

def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Get the content
        if isinstance(doc, Document):
            doc_content = doc.page_content
        else:
            doc_content = doc
        # Ensure doc_content is a string
        if isinstance(doc_content, bytes):
            doc_str = doc_content.decode('utf-8')
        else:
            doc_str = doc_content  # already a string
        if looks_like_base64(doc_str) and is_image_data(doc_str):
            b64_images.append(doc_str)
        else:
            texts.append(doc_str)
    return {"images": b64_images, "texts": texts}

## 5.2. Examine Retrievals (Optional)

In [ ]:
# Check retrieval
query = "Tell me detailed statistics of the top 5 years with largest wildfire acres burned"
docs = retriever_multimodal.invoke(query, limit=5)

# We get 4 docs
len(docs)

In [ ]:
docs

In [ ]:
is_image_data(docs[1].decode('utf-8'))

In [ ]:
r = split_image_text_types(docs)
r

In [ ]:
plt_img_base64(docs[1].decode('utf-8'))

# 6. Construct End-to-End RAG Pipeline

## 6.1. Chain RAG Components Altogether

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import HumanMessage

def multimodal_prompt_function(data_dict):
    """
    Create a multimodal prompt with both text and image context.

    This function formats the provided context from `data_dict`, which contains
    text, tables, and base64-encoded images. It joins the text (with table) portions
    and prepares the image(s) in a base64-encoded format to be included in a message.

    The formatted text and images (context) along with the user question are used to
    construct a prompt for GPT-4o
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            f"""You are an analyst tasked with understanding detailed information and trends from text documents,
                data tables, and charts and graphs in images.
                You will be given context information below which will be a mix of text, tables, and images usually of charts or graphs.
                Use this information to provide answers related to the user question.
                Do not make up answers, use the provided context documents below and answer the question to the best of your ability.

                User question:
                {data_dict['question']}

                Context documents:
                {formatted_texts}

                Answer:
            """
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]

## 6.2. Implement RAG for QA

In [ ]:
# Create RAG chain
rag = (
        {
            "context": itemgetter('context'),
            "question": itemgetter('input'),
        }
            |
        RunnableLambda(multimodal_prompt_function)
            |
        chatgpt
            |
        StrOutputParser()
)

In [ ]:
def rag_qa(query, retriever):

    # Pass input query to retriever and get context document elements
    retrieve_docs = (itemgetter('input')
                        |
                    retriever
                        |
                    RunnableLambda(split_image_text_types))
    
    # Below, we chain `.assign` calls. This takes a dict and successively
    # adds keys-- "context" and "answer"-- where the value for each key
    # is determined by a Runnable (function or chain executing at runtime).
    # This helps in also having the retrieved context along with the answer generated by GPT-4o
    rag_w_sources = (RunnablePassthrough.assign(context=retrieve_docs)
                                        .assign(answer=rag)
    )

    response = rag_w_sources.invoke({'input': query})
    print('=='*50)
    print('Answer:')
    display(Markdown(response['answer']))
    print('--'*50)
    print('Sources:')
    text_sources = response['context']['texts']
    img_sources = response['context']['images']
    for text in text_sources:
        display(Markdown(text))
        print()
    for img in img_sources:
        plt_img_base64(img)
        print()
    print('=='*50)

In [ ]:
def no_rag_qa(query):
    # Create a prompt that only includes the user's question
    messages = [
        HumanMessage(content=f"You are an assistant that answers questions based solely on the input provided.\n\nUser question:\n{query}\n\nAnswer:")
    ]
    
    response = chatgpt.invoke(messages)
    print('=='*50)
    print('Answer:')
    display(Markdown(response.content))
    print('=='*50)

## 6.3. Examine Multimodal RAG (Optional)

In [ ]:
query = "Tell me about the annual wildfires trend with acres burned"
rag_qa(query, retriever_multimodal)

In [ ]:
query = "Tell me about the number of acres burned by wildfires for the forest service in 2021"
rag_qa(query, retriever_multimodal)

In [ ]:
query = "Tell me about the percentage of residences burned by wildfires in 2022"
rag_qa(query, retriever_multimodal)

## 6.4. Examine Single-Modal RAG (Optional)

In [ ]:
query = "Tell me about the annual wildfires trend with acres burned"
rag_qa(query, retriever_single_modal)

In [ ]:
query = "Tell me about the number of acres burned by wildfires for the forest service in 2021"
rag_qa(query, retriever_single_modal)

In [ ]:
query = "Tell me about the percentage of residences burned by wildfires in 2022"
rag_qa(query, retriever_single_modal)

## 6.5. Examine No-RAG (Optional)

In [ ]:
query = "Tell me about the annual wildfires trend with acres burned"
no_rag_qa(query)

In [ ]:
query = "Tell me about the number of acres burned by wildfires for the forest service in 2021"
no_rag_qa(query)

In [ ]:
query = "Tell me about the percentage of residences burned by wildfires in 2022"
no_rag_qa(query)

# 7. Query Energy Advisor Exam Questions

## 7.1. Read the Excel File

In [ ]:
import pandas as pd
import re
import base64
import os
from io import BytesIO
from PIL import Image as PILImage
from IPython.display import display, Markdown, HTML

# Path to the Excel file
excel_file_path = './EA QAs/EA QAs.xlsx'

# Read the Excel file into a DataFrame
questions_df = pd.read_excel(excel_file_path, sheet_name='EA QAs')

# Ensure the columns are correct
print(questions_df.columns)

## 7.2. Define Helper Functions

In [ ]:
def extract_answer(text):
    """Extracts the agent's answer, ensuring it is one of (A), (B), (C), or (D)."""
    match = re.search(r'\b([ABCD])\b', text.strip().upper())
    return match.group(1) if match else None

In [ ]:
def encode_image(image_path, max_size=(800, 800)):
    """
    Encodes an image file to a Base64 string after resizing it to a maximum size.
    """
    try:
        with PILImage.open(image_path) as img:
            # Resize the image while maintaining aspect ratio
            img.thumbnail(max_size, PILImage.LANCZOS)
            
            # Save the image to a BytesIO object
            buffered = BytesIO()
            img.save(buffered, format="JPEG", quality=85)
            
            # Encode the image to Base64
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            return img_str
    except FileNotFoundError:
        print(f"Warning: Image file {image_path} not found.")
        return None

In [ ]:
def resize_base64_image(img_base64, max_size=(800, 800)):
    """Resizes a Base64-encoded image to the specified maximum size."""
    img_data = base64.b64decode(img_base64)
    img = PILImage.open(BytesIO(img_data))
    img.thumbnail(max_size, PILImage.LANCZOS)
    buffered = BytesIO()
    img.save(buffered, format="JPEG", quality=85)
    resized_img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return resized_img_str

In [ ]:
def split_docs_into_images_texts_tables(docs):
    """Splits documents into images, texts, and tables."""
    images = []
    texts = []
    tables = []
    for doc in docs:
        # Extract content and metadata
        content = doc.page_content if hasattr(doc, 'page_content') else str(doc)
        
        # Decode content if it's bytes
        if isinstance(content, bytes):
            content = content.decode('utf-8', errors='ignore')
        
        metadata = doc.metadata if hasattr(doc, 'metadata') else {}
        category = metadata.get('category', '').lower()
        
        # Check the category to classify the content
        if content.startswith('data:image'):
            images.append(content)
        elif category == 'table' or '<table' in content.lower():
            tables.append({'content': content, 'metadata': metadata})
        else:
            texts.append(content)
    return {'images': images, 'texts': texts, 'tables': tables}

In [ ]:
def display_base64_image(img_base64):
    """Displays a Base64-encoded image."""
    img_data = base64.b64decode(img_base64)
    img = PILImage.open(BytesIO(img_data))
    display(img)

## 7.3. Modify RAG Functions

In [ ]:
def rag_multiple_choice_qa(question, options, retriever, image_base64=None):
    """Answers a multiple-choice question using the given retriever."""
    # Retrieve relevant documents
    retrieved_docs = retriever.get_relevant_documents(question)
    
    # Split documents into images, texts, and tables
    sources = split_docs_into_images_texts_tables(retrieved_docs)
    
    # Limit text sources to 100 words
    def limit_text_length(text):
        words = text.split()
        return ' '.join(words[:100])
    
    sources['texts'] = [limit_text_length(text) for text in sources['texts']]
    
    # Build the prompt
    formatted_texts = "\n".join(sources['texts'])
    messages = []
    
    # Include the question image in the messages if provided
    if image_base64:
        # Resize the question image if necessary
        resized_image_base64 = resize_base64_image(image_base64)
        messages.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{resized_image_base64}"}
        })
    
    # Add images from retrieved sources to the messages
    for image_data in sources['images']:
        # Extract the Base64 data
        img_base64 = image_data.split(',', 1)[1] if ',' in image_data else image_data
        # Resize the image
        resized_img_base64 = resize_base64_image(img_base64)
        # Reconstruct the data URL
        resized_image_data = f"data:image/jpeg;base64,{resized_img_base64}"
        messages.append({
            "type": "image_url",
            "image_url": {"url": resized_image_data},
        })
    
    # Add tables from retrieved sources to the messages
    for table_dict in sources['tables']:
        table_content = table_dict['content']
        # Decode table_content if it's bytes
        if isinstance(table_content, bytes):
            table_content = table_content.decode('utf-8', errors='ignore')
        messages.append({
            "type": "text",
            "text": table_content  # Include the table content as text (HTML)
        })
    
    # Add the text message
    prompt_text = f"""You are an assistant that answers multiple-choice questions based solely on the input provided.
You will be given a question, several options, and context documents, which may include text, HTML tables, and images.
Use the context documents to choose the correct option.
Respond with only the letter of the chosen option: (A), (B), (C), or (D).

User question:
{question}

Options:
{options}

Context documents:
{formatted_texts}

Answer:"""
        
    messages.append({
        "type": "text",
        "text": prompt_text
    })
    
    # Send the messages to the LLM
    response = chatgpt.invoke([HumanMessage(content=messages)])
    
    # Extract the answer
    answer_text = response.content.strip()
    answer = extract_answer(answer_text)
    
    return answer, sources

In [ ]:
def no_rag_multiple_choice_qa(question, options, image_base64=None):
    """Answers a multiple-choice question without retrieval (no RAG)."""
    messages = []
    
    # Include the question image in the messages if provided
    if image_base64:
        # Resize the question image if necessary
        resized_image_base64 = resize_base64_image(image_base64)
        messages.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{resized_image_base64}"}
        })
    
    # Add the question and options
    prompt_text = f"""You are an assistant that answers multiple-choice questions based solely on the input provided.
You will be given a question and several options.
Choose the correct option from the given options, and respond with only the letter of the chosen option: (A), (B), (C), or (D).

User question:
{question}

Options:
{options}

Answer:"""
    
    messages.append({
        "type": "text",
        "text": prompt_text
    })
    
    # Send the messages to the LLM
    response = chatgpt.invoke([HumanMessage(content=messages)])
    
    # Extract the answer
    answer_text = response.content.strip()
    answer = extract_answer(answer_text)
    
    return answer

## 7.4. Process Each Question and Collect Responses

In [ ]:
# Main loop to process the questions
for idx, row in questions_df.iterrows():
    question_number = row['#']
    question_text = row['Question']
    image_needed = row['Image']
    options_text = row['Option']
    true_answer = row['Answer']

    # Load image if needed
    image_base64 = None
    if str(image_needed).strip().upper() == 'TRUE':
        image_path = os.path.join('EA QAs', f"{question_number}.jpg")
        image_base64 = encode_image(image_path, max_size=(800, 800))  # Adjust size as needed

    # Multimodal RAG agent
    mm_answer, mm_sources = rag_multiple_choice_qa(
        question_text, options_text, retriever_multimodal, image_base64
    )

    # Single-modal RAG agent (no images in context)
    sm_answer, sm_sources = rag_multiple_choice_qa(
        question_text, options_text, retriever_single_modal, image_base64=None
    )

    # No-RAG agent
    nr_answer = no_rag_multiple_choice_qa(
        question_text, options_text, image_base64
    )

    # Compare answers to true answer
    mm_correct = 'CORRECT' if mm_answer == true_answer else 'WRONG'
    sm_correct = 'CORRECT' if sm_answer == true_answer else 'WRONG'
    nr_correct = 'CORRECT' if nr_answer == true_answer else 'WRONG'

    # Display the results
    print(f"[{question_number}]")
    print("-"*30)
    print(f"({true_answer}): True answer\n")
    print(f"({mm_answer}): Multimodal RAG - {mm_correct}")
    print(f"({sm_answer}): Single-modal RAG - {sm_correct}")
    print(f"({nr_answer}): No-RAG - {nr_correct}")
    print("-"*30)
    print("Question:")
    print(question_text)
    if image_base64:
        display_base64_image(image_base64)
    print("\nOptions:")
    print(options_text)

    '''
    print("-"*30)
    print("Multimodal RAG:")
    for i, text in enumerate(mm_sources['texts']):
        print(f"Source {i+1}:")
        display(Markdown(text))
        print()
    for i, img_data in enumerate(mm_sources['images']):
        print(f"Image Source {i+1}:")
        # Extract Base64 data
        img_base64 = img_data.split(',', 1)[1] if ',' in img_data else img_data
        # Resize the image for display
        resized_img_base64 = resize_base64_image(img_base64)
        display_base64_image(resized_img_base64)
        print()
    # Display tables if any
    for i, table_dict in enumerate(mm_sources['tables']):
        print(f"Table Source {i+1}:")
        table_content = table_dict['content']
        # Decode table_content if it's bytes
        if isinstance(table_content, bytes):
            table_content = table_content.decode('utf-8', errors='ignore')
        display(HTML(table_content))
        print()
    print("-"*30)
    print("Single-modal RAG:")
    for i, text in enumerate(sm_sources['texts']):
        print(f"Source {i+1}:")
        display(Markdown(text))
        print()
    # No images or tables for single-modal RAG
    '''
    
    print("="*40)